In [ ]:
pip install mediapipe

In [ ]:
pip install opencv-python

In [ ]:
pip install opencv-contrib-python

In [6]:
import cv2
import mediapipe as mp 
import time

cap = cv2.VideoCapture(1)
 
pTime = 0

# 얼굴 detect
mpFaceDect = mp.solutions.face_detection
mpDrawing = mp.solutions.drawing_utils
faceDetection = mpFaceDect.FaceDetection(0.75)

# 손 detect
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

cnt = 0 # 프레임별 바로바로 카운팅
# cnt가 일정 기준값 이상일때(지금은 일단 60) 옐로카드 1씩 증가
# -> 예를 들어 5이상 되면 레드카드 한장 이런식으로 정의 가능
yellow_card = 0

with mp_hands.Hands(max_num_hands=4,min_detection_confidence=0.4,min_tracking_confidence=0.4) as hands:
    while True:
        success, img = cap.read()
        hand_num = 0
        face_num = 0

        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        
        # 얼굴 결과
        results_face = faceDetection.process(img)
        
        # 손 결과
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        #img = cv2.flip(img, 1)
        img.flags.writeable = False
        results_hand = hands.process(img)
        # Draw the hand annotations on the image.
        img.flags.writeable = True
        
        if results_face.detections:
            for id, detection in enumerate(results_face.detections):
                # 얼굴 그리기
                mpDrawing.draw_detection(img, detection)
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, ic = img.shape
                bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), \
                    int(bboxC.width * iw), int(bboxC.height * ih)
                cv2.rectangle(img, bbox, (255, 0, 255), 2)
                cv2.putText(img, f'{int(detection.score[0] * 100)}%',
                           (bbox[0], bbox[1] - 20),cv2.FONT_HERSHEY_PLAIN,2,(255, 0, 255),2)
                face_num = face_num + 1

            # 얼굴이 두개 이상일 때
            if face_num > 1:
                cnt = cnt + 1
            face_num = 0
        # 얼굴이 하나도 감지 안될 때
        else:
            cnt = cnt + 1
            
        if results_hand.multi_hand_landmarks:
                # 손 그리기
            for hand_landmarks in results_hand.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    img,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style())
                hand_num = hand_num + 1
            
            # 손이 두개가 아닐 때
            if hand_num != 2:
                cnt = cnt + 1
        # 손이 하나도 감지 안될 때        
        else:
            cnt = cnt + 1
        
        # 옐로카드 카운팅
        if cnt != 0 and cnt % 120 == 0:
            yellow_card = yellow_card + 1
            print("옐로카드", yellow_card)

        cTime = time.time() # frame rate
        fps_rate = 1 / (cTime-pTime)
        pTime = cTime

        cv2.putText(img, f'FPS: {int(fps_rate)}', (20, 70), cv2.FONT_HERSHEY_PLAIN,3  ,(0, 255, 0), 2)
        cv2.imshow('Face and Hand Detection ', img)
        # cv2.waitKey(5)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
        
cap.release()
cv2.destroyAllWindows()

In [5]:
cap.release()
cv2.destroyAllWindows()